In [1]:
import pandas as pd

df=pd.read_csv("csvs/2019-04-01.csv")

In [2]:
for i in range(2,8):
    print("csvs/2019-04-{}.csv".format('0'+str(i) if i<10 else str(i)))
    temp=pd.read_csv("csvs/2019-04-{}.csv".format('0'+str(i) if i<10 else str(i)), low_memory=False)
    df=pd.concat([df,temp])

csvs/2019-04-02.csv
csvs/2019-04-03.csv
csvs/2019-04-04.csv
csvs/2019-04-05.csv
csvs/2019-04-06.csv
csvs/2019-04-07.csv


In [3]:
df.inventory_source.unique()

array(['MOPUB', 'GOOGLE_ADX'], dtype=object)

In [4]:
df.creative_type.unique()

array(['video', 'banner', nan], dtype=object)

In [5]:
X=df[['inventory_source','creative_type','hour','bid_floor', 'rewarded']].values
X

array([['MOPUB', 'video', '19:00', 0.00337, 1.0],
       ['MOPUB', 'video', '19:00', 0.00337, 1.0],
       ['MOPUB', 'video', '19:00', 0.00337, 1.0],
       ...,
       ['MOPUB', nan, '11:00', 0.0020800000000000003, 0.0],
       ['MOPUB', nan, '13:00', 4e-05, 0.0],
       ['MOPUB', nan, '15:00', 0.00085, 0.0]], dtype=object)

In [6]:
y=df['clicks'].values
y

array([0., 0., 0., ..., 0., 0., 0.])

In [7]:
for i in range(len(X)):
    if X[i][0]=='MOPUB':
        X[i][0]=1
    elif X[i][0]=='GOOGLE_ADX':
        X[i][0]=2
    else:
        X[i][0]=0
    
    if X[i][1]=='video':
        X[i][1]=1
    elif X[i][1]=='banner':
        X[i][1]=2
    else:
        X[i][1]=0
        
    X[i][2]=int(str(X[i][2][:2]))
    
X

array([[1, 1, 19, 0.00337, 1.0],
       [1, 1, 19, 0.00337, 1.0],
       [1, 1, 19, 0.00337, 1.0],
       ...,
       [1, 0, 11, 0.0020800000000000003, 0.0],
       [1, 0, 13, 4e-05, 0.0],
       [1, 0, 15, 0.00085, 0.0]], dtype=object)

In [8]:
from imblearn.under_sampling import RandomUnderSampler

X_resampled, y_resampled = RandomUnderSampler(sampling_strategy='majority').fit_resample(X, y)

In [9]:
X_resampled

array([[1, 0, 1, 0.00419, 0.0],
       [1, 0, 20, 0.00505, 0.0],
       [1, 0, 2, 0.00085, 0.0],
       ...,
       [1, 0, 13, 0.0020800000000000003, 0.0],
       [1, 0, 10, 0.0017699999999999999, 0.0],
       [1, 0, 13, 0.00385, 0.0]], dtype=object)

In [10]:
y_resampled

array([0., 0., 0., ..., 1., 1., 1.])

In [11]:
len(X_resampled), len(X)

(285860, 5021077)

In [12]:
len(y_resampled), len(y)

(285860, 5021077)

In [13]:
import pandas as pd

test=pd.read_csv("csvs/2019-04-15.csv", low_memory=False)

In [14]:
test.inventory_source.unique()

array(['PUBMATIC', 'RUBICON', 'MOPUB', 'GOOGLE_ADX'], dtype=object)

In [15]:
test.creative_type.unique()

array([nan, 'video', 'banner'], dtype=object)

In [16]:
X_test=test[['inventory_source','creative_type','hour','bid_floor','rewarded']].values
X_test

array([['PUBMATIC', nan, '21:00', 0.000361, 0.0],
       ['RUBICON', nan, '21:00', 0.000411, 0.0],
       ['PUBMATIC', nan, '21:00', 0.000337, 0.0],
       ...,
       ['MOPUB', 'banner', '16:00', 0.00139, 0.0],
       ['MOPUB', 'banner', '16:00', 0.0016899999999999999, 0.0],
       ['MOPUB', nan, '16:00', 0.00085, 0.0]], dtype=object)

In [17]:
y_test=test['clicks'].values
y_test

array([0., 0., 0., ..., 0., 0., 0.])

In [18]:
len(X_test), len(y_test), sum(y_test)

(982394, 982394, 74299.0)

In [19]:
for i in range(len(X_test)):
    if X_test[i][0]=='MOPUB':
        X_test[i][0]=1
    elif X_test[i][0]=='GOOGLE_ADX':
        X_test[i][0]=2
    else:
        X_test[i][0]=0
    
    if X_test[i][1]=='video':
        X_test[i][1]=1
    elif X_test[i][1]=='banner':
        X_test[i][1]=2
    else:
        X_test[i][1]=0
        
    X_test[i][2]=int(str(X_test[i][2][:2]))

    
X_test

array([[0, 0, 21, 0.000361, 0.0],
       [0, 0, 21, 0.000411, 0.0],
       [0, 0, 21, 0.000337, 0.0],
       ...,
       [1, 2, 16, 0.00139, 0.0],
       [1, 2, 16, 0.0016899999999999999, 0.0],
       [1, 0, 16, 0.00085, 0.0]], dtype=object)

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, balanced_accuracy_score

LR=LogisticRegression(solver='lbfgs')
LR.fit(X_resampled,y_resampled)
y1 = LR.predict(X_test)
print(classification_report(y_test, y1))
f1_score(y_test, y1), balanced_accuracy_score(y_test, y1)

              precision    recall  f1-score   support

         0.0       0.93      0.54      0.68    908095
         1.0       0.08      0.52      0.14     74299

   micro avg       0.54      0.54      0.54    982394
   macro avg       0.51      0.53      0.41    982394
weighted avg       0.87      0.54      0.64    982394



(0.14409683175145815, 0.5269952110745642)

In [29]:
len(y1),sum(y1),sum(y_test),sum(abs(y1-y_test))

(982394, 459245.0, 74299.0, 456662.0)

In [33]:
from sklearn.ensemble import BaggingClassifier

bc = BaggingClassifier(random_state=10000)

bc.fit(X_resampled, y_resampled) 
y2 = bc.predict(X_test)
print(classification_report(y_test, y2))
f1_score(y_test, y2), balanced_accuracy_score(y_test, y2)

              precision    recall  f1-score   support

         0.0       0.94      0.39      0.55    908095
         1.0       0.08      0.69      0.15     74299

   micro avg       0.41      0.41      0.41    982394
   macro avg       0.51      0.54      0.35    982394
weighted avg       0.87      0.41      0.52    982394



(0.15010508672397074, 0.538032170340941)

In [34]:
len(y2),sum(y2),sum(y_test),sum(abs(y2-y_test))

(982394, 609422.0, 74299.0, 581091.0)

In [37]:
from sklearn import ensemble

GBclf = ensemble.GradientBoostingClassifier(learning_rate=0.001)
GBclf.fit(X_resampled,y_resampled)
y3=GBclf.predict(X_test)
print(classification_report(y_test, y3))
f1_score(y_test, y3), balanced_accuracy_score(y_test, y3)

              precision    recall  f1-score   support

         0.0       0.93      0.54      0.68    908095
         1.0       0.08      0.52      0.14     74299

   micro avg       0.54      0.54      0.54    982394
   macro avg       0.51      0.53      0.41    982394
weighted avg       0.87      0.54      0.64    982394



(0.14455153219057648, 0.5279219988230657)

In [38]:
len(y3),sum(y3),sum(y_test),sum(abs(y3-y_test))

(982394, 457456.0, 74299.0, 454889.0)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ABclf=AdaBoostClassifier()
ABclf.fit(X_resampled, y_resampled)
y4=ABclf.predict(X_test)
print(classification_report(y_test, y4))
f1_score(y_test, y4), balanced_accuracy_score(y_test, y4)

In [ ]:
len(y4),sum(y4),sum(y_test),sum(abs(y4-y_test))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RFclf=RandomForestClassifier()
RFclf.fit(X_resampled, y_resampled)
y5=RFclf.predict(X_test)
print(classification_report(y_test, y5))
f1_score(y_test, y5), balanced_accuracy_score(y_test, y5)

In [ ]:
len(y5),sum(y5),sum(y_test),sum(abs(y5-y_test))

In [ ]:
from sklearn.ensemble import VotingClassifier

Vclf=VotingClassifier(
    estimators=[('Bagging', bc), ('GradientBoost', GBclf), ('RandomForest', RFclf)],
    voting='soft')
Vclf.fit(X_resampled, y_resampled)
y_pred=Vclf.predict(X_test)
print(classification_report(y_test, y_pred))
f1_score(y_test, y_pred), balanced_accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.neural_network import MLPClassifier

MLPclf = MLPClassifier(solver='lbfgs', alpha=1e-8, activation='logistic', 
                       hidden_layer_sizes=(5, 4), max_iter=1000)

MLPclf.fit(X_resampled, y_resampled)
y6=MLPclf.predict(X_test)
print(classification_report(y_test, y6))
f1_score(y_test, y6), balanced_accuracy_score(y_test, y6)


In [ ]:
len(y6),sum(y6),sum(y_test),sum(abs(y6-y_test))